First assemble a whole big dataset named `small_passenge.json`, using the [Nvidia dataset](https://huggingface.co/datasets/nvidia/ChatQA-Training-Data). That datasets include files like drop/train.json, narrativeqa/train.json, etc, which will be used below. Modify their file paths to your need.



In [ ]:
import json
import pandas as pd

empty_df = pd.DataFrame()
files=["ChatQA-Training-Data/drop/train.json",
       "ChatQA-Training-Data/narrativeqa/train.json",
       "ChatQA-Training-Data/quoref/train.json",
       "ChatQA-Training-Data/newsqa/train.json",
       "ChatQA-Training-Data/ropes/train.json",
       "ChatQA-Training-Data/squad1.1/train.json",
       "ChatQA-Training-Data/squad2.0/train.json",
       "ChatQA-Training-Datat/synthetic_convqa/train.json",
       "ChatQA-Training-Data/tatqa/train_arithmetic.json",
       "ChatQA-Training-Data/tatqa/train_others.json"]
for file in files:
    data = pd.read_json(file)
    data_unique = data.drop_duplicates(subset=['document'])
    data_unique=data_unique[['document']]
    empty_df= pd.concat([empty_df, data_unique])
empty_df = empty_df.drop_duplicates(subset=['document'])

empty_df.to_json('small_passenge.json', orient='records', indent=4, date_format='iso')


Now generate our chunking dataset `mixed_passenge_train_30000.json`. We sample from small_passenge.json 4 different rows to cross them into one row of chunking dataset `mixed_passenge_train_30000.json`, which they are separated by symbol "<--start-->". We generate 30000 of rows for chunking dataset. 

In [4]:
START_SYM="<--start-->"
file="/data/bert-chunker-v2/small_passenge.json"
import pandas as pd
data = pd.read_json(file)
empty_df = pd.DataFrame()
background_sentence_num=4
avrge_tokens_per_sentence=20
max_tokens=256

import random
import nltk
from tqdm import tqdm
# nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

for i in tqdm(range(0, 30000)):
    sam = data.sample(n=4)
    res = ""

    for index, row in sam.iterrows():

        sentence_num = random.randint(1, 5)
        ss = sent_tokenize(row["document"])[0:sentence_num]
        ss = "".join(ss)
        res = res + START_SYM + ss

    res = res[11:]
    sam = pd.DataFrame({"input": [res]})
    empty_df = pd.concat([empty_df, sam])

empty_df = empty_df.drop_duplicates(subset=["input"])
empty_df.to_json(
    "/data/mixed_passenge_train_30000.json",
    orient="records",
    indent=4,
    date_format="iso",
)

KeyboardInterrupt: 

Run the same code but to generate `mixed_passenge_eval_30000.json` for evaluation.

下面是临时的模型融合代码

In [54]:
import torch
from safetensors.torch import load_file, save_file

# 加载两个模型的权重
weights1 = load_file("/data/bert-chunker-v2/model_soup/huanhang-model.safetensors")
weights2 = load_file("/data/bert-chunker-v2/model_soup/yuyi-model.safetensors")
weights2 = load_file("/data/tim1900/bert-chunker/model.safetensors")


# 定义权重
w1, w2 = 0.9,0.1  # 可以根据需要调整

# 创建一个字典来保存新的加权平均权重
new_weights = {}

# 遍历所有参数并计算加权平均
for key in weights1.keys():
    if key in weights2:  # 确认两个模型都有这个参数
        new_weights[key] = (w1 * weights1[key] + w2 * weights2[key]) / (w1 + w2)

# 保存新的权重到一个新的 .safetensors 文件
save_file(new_weights, "/data/bert-chunker-v2/model_soup/model.safetensors")